# Logistic Regression

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt

# 0️⃣ Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape # returns rows, columns
print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale the features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Reshape the data
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

n_samples, n_features = X.shape

# 1️⃣ Design model
input_size = n_features
output_size = 1
class LogisticRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    y_predicted = self.linear(x)
    y_predicted = torch.sigmoid(y_predicted)
    return y_predicted

model = LogisticRegression(input_size, output_size,)

# 2️⃣ Define loss and optimizer
criterion = nn.BCELoss() #Binary Cross Entropy Loss
learning_rate = 0.01
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.ASGD(model.parameters(), lr=learning_rate)

# ⚙️ 3️⃣ Training loop
epochs = 100
for epoch in range(epochs):
    # forward pass and loss calculation
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    # backward pass
    loss.backward()

    # update weights
    optimizer.step()

    # reset gradients
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w, b] = model.parameters()
        print(f"Epoch {epoch+1:03d}: loss = {loss.item():.8f}")

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    accuracy = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"Training accuracy: {accuracy}")


569 30
Epoch 001: loss = 0.56373078
Epoch 011: loss = 0.46830007
Epoch 021: loss = 0.40680563
Epoch 031: loss = 0.36377230
Epoch 041: loss = 0.33178979
Epoch 051: loss = 0.30693895
Epoch 061: loss = 0.28696585
Epoch 071: loss = 0.27048442
Epoch 081: loss = 0.25659579
Epoch 091: loss = 0.24469092
Training accuracy: 0.8771929740905762
